# Artificial Neural Networks and Deep Learning

---

## Homework 1: Minimal Working Example

To make your first submission, follow these steps:
1. Create a folder named `[2024-2025] AN2DL/Homework 1` in your Google Drive.
2. Upload the `training_set.npz` file to this folder.
3. Upload the Jupyter notebook `Homework 1 - Minimal Working Example.ipynb`.
4. Load and process the data.
5. Implement and train your model.
6. Submit the generated `.zip` file to Codabench.


## ⚙️ Import Libraries

In [ ]:
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
import json
import numpy as np
import random
import seaborn as sns
import tensorflow as tf

In [ ]:
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
# Define if we wanna assign different class weights (for class imbalance) during model fitting
USE_CLASS_WEIGHTS = False

## ⏳ Load and inspect the data

In [ ]:
# Define img shape. Input image is 96x96 hence based on the specified value it will be enlarged or CENTER cropped
IMG_SIZE = 64

if IMG_SIZE < 96:
	print('Image will center cropped!')
elif IMG_SIZE > 96:
	print('Image will enlarged!')

In [ ]:
DATASET = "training_set.npz"
OUTLIERS = "training-data-filter/blacklist.json"

In [ ]:
# TODO: maybe adjust
train_ratio = 0.64
validation_ratio = 0.24
test_ratio = 1.0 - train_ratio - validation_ratio


In [ ]:
data = np.load(DATASET)
X = data['images']
y = data['labels']

X = (X).astype('float32')

print('Before data points filter shape:', X.shape, y.shape)

with open(OUTLIERS, 'r') as file:
	blacklist = json.load(file)
blacklist = sorted(blacklist['blacklist'])
X = np.delete(X, blacklist, axis=0)
y = np.delete(y, blacklist, axis=0)

print('After data points filter shape:', X.shape, y.shape)

# Percentages taken from:  https://arxiv.org/pdf/2110.09508
train_size = int(X.shape[0] * train_ratio)
val_size = int(X.shape[0] * validation_ratio)
test_size = X.shape[0] - train_size - val_size

if not USE_CLASS_WEIGHTS:
	# Convert to one hoot encoding
	y = tfk.utils.to_categorical(y)

	X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed, test_size=test_size, stratify=y)
	X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=seed, test_size=val_size, stratify=y_train)
	print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)
else:
	X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed, test_size=test_size, stratify=y)
	X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=seed, test_size=val_size, stratify=y_train)
	# Copy for later
	y_train_cat = y_train
	# Convert to one hoot encoding
	y_train = tfk.utils.to_categorical(y_train)
	y_val = tfk.utils.to_categorical(y_val)
	y_test = tfk.utils.to_categorical(y_test)
	print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

In [ ]:
# Labels
labels = {
	0: "Basophil",
	1: "Eosinophil",
	2: "Erythroblast",
	3: "Immature granulocytes",
	4: "Lymphocyte",
	5: "Monocyte",
	6: "Neutrophil",
	7: "Platelet"
}

In [ ]:
# Inspect data
# Display a sample of images from the training-validation dataset
num_img = 10
random_indices = random.sample(range(len(X_val)), num_img)

fig, axes = plt.subplots(1, num_img, figsize=(20, 20))

def get_label(y):
    index = np.where(y == 1)[0]
    return labels[int(index)]

# Iterate through the selected number of images
for i, idx in enumerate(random_indices):
    ax = axes[i % num_img]
    ax.imshow(np.squeeze(X_val[idx] / 255), vmin=0., vmax=1.)
    ax.set_title(get_label(y_val[idx]))
    ax.axis('off')

# Adjust layout and display the images
plt.tight_layout()
plt.show()

## 🌊 Generate class weights

In [ ]:
if USE_CLASS_WEIGHTS:
    # Flat the train labels
    y_train_cat_flat = np.ravel(y_train_cat)

    # Make weights proportional to class imbalance

    class_weights = compute_class_weight(
        class_weight='balanced', 
        classes=np.unique(data['labels']), 
        y=y_train_cat_flat
    )

    class_weight_dict = dict(enumerate(class_weights))
    from pprint import pprint
    print('Class weights:')
    pprint(class_weight_dict)

## 🛠️ Train and Save the Model

In [ ]:
# Define training setup
epochs = 400
batch_size = 64

In [ ]:
# Define optimizer setup
lr = 1e-3
fine_tuning_lr = 1e-4
opt_name = "SGD"
fine_tuning_opt_name = "Lion"

opt_exp_decay_rate: int | None = 0.1
# Decay at how many epochs
opt_decay_epoch_delta = 7

In [ ]:

# Define dense params
dropouts_layers = [0.4]
# Note, the base model outputs a size which is different based on the model being used, hence make attention on the first dense size
dense_layers = [8]

# Example for more layers:
#dropouts_layers = [0.5, 0.3, 0.4]
#dense_layers = [256, 64, 8]

In [ ]:
# just to free or not the memory
FREE_MODEL = False

In [ ]:
# Base model name being used. One of
# vgg19
# vgg16
# effnetV2B0
# effnetV2B3
# effnetV2S
# effnetV2M
# effnetV2L
# They all use global average pooling
base_model_name = 'vgg19'

In [ ]:
# Define if to load a trained classier based on the same base model
LOAD_TRAINED_CLASSIFIER = False
trained_classifier_model_file = ""

In [ ]:
# Layers to fine tune
based_model_layers_to_activate = set([
  # These are vgg19 layers
  "block5_conv4",
  "block5_conv3",
  "block5_conv2",
])

In [ ]:
base_model_dict = {
  'vgg19': tfk.applications.VGG19(
      include_top=False,
      input_shape=(IMG_SIZE, IMG_SIZE, 3),
      input_tensor=None,
      pooling='avg',
      weights="imagenet",
  ),
  'vgg16': tfk.applications.VGG16(
      include_top=False,
      input_shape=(IMG_SIZE, IMG_SIZE, 3),
      input_tensor=None,
      pooling='avg',
      weights="imagenet",
  ),
  'effnetV2L': tfk.applications.EfficientNetV2L(
    include_preprocessing=True,
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    input_tensor=None,
    pooling='avg',
    weights="imagenet",
  ),
  'effnetV2S': tfk.applications.EfficientNetV2S(
    include_preprocessing=True,
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    input_tensor=None,
    pooling='avg',
    weights="imagenet",
  ),
  'effnetV2M': tfk.applications.EfficientNetV2M(
    include_preprocessing=True,
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    input_tensor=None,
    pooling='avg',
    weights="imagenet",
  ),
  'effnetV2B0': tfk.applications.EfficientNetV2B0(
    include_preprocessing=True,
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    input_tensor=None,
    pooling='avg',
    weights="imagenet",
  ),
  'effnetV2B3': tfk.applications.EfficientNetV2B3(
    include_preprocessing=True,
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    input_tensor=None,
    pooling='avg',
    weights="imagenet",
  ),

}

def get_base_model():
  if LOAD_TRAINED_CLASSIFIER:
    return None
  # Initialise imageNet model with pretrained weights, for transfer learning
  assert(base_model_name in base_model_dict)
  m = base_model_dict[base_model_name]

  # Freeze
  m.trainable = False
  return m

In [ ]:
# TODO
def build_augmentation(name = 'preprocessing'):
	augmentation = tf.keras.Sequential([
	    tfkl.RandomFlip("horizontal_and_vertical"),
	    tfkl.RandomRotation(0.167), # 60%
	    tfkl.CenterCrop(IMG_SIZE, IMG_SIZE),
	], name=name)
	return augmentation

In [ ]:
# By default we build a new model with the given base model
# If the flag LOAD_TRAINED_CLASSIFIER is True, we load the model and load the base_model from the loaded model, hence any base_model being passed is ignored
def build_model(base_model = None, restore_base = True, out_shape = y_train.shape[-1], trained_classifier_model_name = 'vgg19'):
	assert(len(dropouts_layers) == len(dense_layers))
	assert(dense_layers[-1] == len(labels))

	if LOAD_TRAINED_CLASSIFIER:
		assert(trained_classifier_model_file != '')
		m = tf.keras.models.load_model(trained_classifier_model_file)

		# Extract the base model
		if restore_base:
			base_model = m.get_layer(trained_classifier_model_name)
			for l in base_model.layers:
				l.trainable = False
			return m

	if restore_base and base_model is not None:
		for l in base_model.layers:
			l.trainable = False

	inputs = tfk.Input(shape=X_train[0].shape, name='input_layer')
	# Define augmentation layers
	augmentation = build_augmentation()
	# Define network
	x = augmentation(inputs)
	if base_model is not None:
		x = base_model(x)
	for i, (drop, dense) in enumerate(zip(dropouts_layers, dense_layers)):
		x = tfkl.Dropout(drop, name=f'dropout{i}')(x)
		# Skip last dense as it's the output
		if i == len(dropouts_layers)-1:
			break
		x = tfkl.Dense(dense, activation='relu', name=f'dense{i}')(x)
	outputs = tfkl.Dense(dense_layers[-1], activation='softmax', name=f'dense{len(dense_layers)-1}')(x)

	# Define the complete model linking input and output
	m = tfk.Model(inputs=inputs, outputs=outputs, name='model')
	return m

In [ ]:
def fit_model(model):
	if USE_CLASS_WEIGHTS:
		print('Fitting with class weights!')
		fit_history = model.fit(
	    x=X_train,
	    y=y_train,
	    batch_size=batch_size,
	    epochs=epochs,
	    validation_data=(X_val, y_val),
	    class_weight = class_weight_dict,
	    callbacks=[tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)]
	  ).history
	else:
		fit_history = model.fit(
	    x=X_train,
	    y=y_train,
	    batch_size=batch_size,
	    epochs=epochs,
	    validation_data=(X_val, y_val),
	    callbacks=[tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)]
	  ).history
	return fit_history

In [ ]:
def enable_feature_extractor_layers(extractor, layers):
  extractor.trainable = True
  for i, layer in enumerate(extractor.layers):
    layer.trainable = False
	# Set the based_model_layers_to_activate layers as trainable
  for i, layer in enumerate(extractor.layers):
    if layer.name in layers:
      layer.trainable = True

	# Print layer indices, names, and trainability status
  print('\n\nBase model training configuration:')
  for i, layer in enumerate(extractor.layers):
	  print(i, layer.name, layer.trainable)

In [ ]:
def get_optimizer(is_fine_tuning = False, use_decay_fine_tuning = False, **kwargs):
	decay = opt_exp_decay_rate
	if is_fine_tuning and not use_decay_fine_tuning:
		decay = None

	opt = opt_name if not is_fine_tuning else fine_tuning_opt_name

	if opt == "SGD":
		optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9 if 'momentum' not in kwargs else kwargs['momentum'])
		if decay is not None:
			lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
					initial_learning_rate=fine_tuning_lr if is_fine_tuning else lr,
					decay_steps=opt_decay_epoch_delta * (X_train.shape[0] // batch_size),  # Decay every 7 epochs
					decay_rate=opt_exp_decay_rate,
					staircase=True
			)
			optimizer.learning_rate = lr_schedule
			print(f'\n\n{"Finetuning: " if is_fine_tuning else "NotFinetuning: "}using SGD optimizer with exp decay {decay} (momentum = {optimizer.momentum})\n\n')
			return optimizer
		else:
			optimizer.learning_rate = fine_tuning_lr if is_fine_tuning else lr
			print(f'\n\n{"Finetuning: " if is_fine_tuning else "NotFinetuning: "}using SGD optimizer (momentum = {optimizer.momentum})\n\n')
			return optimizer

	elif opt == "Adam":
		if 'weight_decay' in kwargs:
			optimizer = tf.keras.optimizers.Adam(weight_decay=kwargs['weight_decay'])
		else:
			optimizer = tf.keras.optimizers.Adam()
		if decay is not None:
			lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
					initial_learning_rate=fine_tuning_lr if is_fine_tuning else lr,
					decay_steps=opt_decay_epoch_delta * (X_train.shape[0] // batch_size),  # Decay every 7 epochs
					decay_rate=opt_exp_decay_rate,
					staircase=True
			)
			optimizer.learning_rate = lr_schedule
			print(f'\n\n{"Finetuning: " if is_fine_tuning else "NotFinetuning: "}using Adam optimizer with exp decay of {decay} weight decay = {optimizer.weight_decay}\n\n')
			return optimizer
		else:
			optimizer.learning_rate = fine_tuning_lr if is_fine_tuning else lr
			print(f'\n\n{"Finetuning: " if is_fine_tuning else "NotFinetuning: "}using Adam optimizer (weight decay = {optimizer.weight_decay})\n\n')
			return optimizer

	elif opt == "AdamW":
		if 'weight_decay' in kwargs:
			optimizer = tf.keras.optimizers.AdamW(weight_decay=kwargs['weight_decay'])
		else:
			optimizer = tf.keras.optimizers.AdamW()
		if decay is not None:
			lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
					initial_learning_rate=fine_tuning_lr if is_fine_tuning else lr,
					decay_steps=opt_decay_epoch_delta * (X_train.shape[0] // batch_size),  # Decay every 7 epochs
					decay_rate=opt_exp_decay_rate,
					staircase=True
			)
			optimizer.learning_rate = lr_schedule
			print(f'\n\n{"Finetuning: " if is_fine_tuning else "NotFinetuning: "}using AdamW optimizer with exp decay of {decay} weight decay = {optimizer.weight_decay}\n\n')
			return optimizer
		else:
			optimizer.learning_rate = fine_tuning_lr if is_fine_tuning else lr
			print(f'\n\n{"Finetuning: " if is_fine_tuning else "NotFinetuning: "}using AdamW optimizer (weight decay = {optimizer.weight_decay})\n\n')
			return optimizer

	elif opt == "Lion":
		if 'weight_decay' in kwargs:
			optimizer = tf.keras.optimizers.Lion(weight_decay=kwargs['weight_decay'])
		else:
			optimizer = tf.keras.optimizers.Lion()
		if decay is not None:
			lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
					initial_learning_rate=fine_tuning_lr if is_fine_tuning else lr,
					decay_steps=opt_decay_epoch_delta * (X_train.shape[0] // batch_size),  # Decay every 7 epochs
					decay_rate=opt_exp_decay_rate,
					staircase=True
			)
			optimizer.learning_rate = lr_schedule
			print(f'\n\n{"Finetuning: " if is_fine_tuning else "NotFinetuning: "}using Lion optimizer with exp decay of {decay} weight decay = {optimizer.weight_decay}\n\n')
			return optimizer
		else:
			optimizer.learning_rate = fine_tuning_lr if is_fine_tuning else lr
			print(f'\n\n{"Finetuning: " if is_fine_tuning else "NotFinetuning: "}using Lion optimizer (weight decay = {optimizer.weight_decay})\n\n')
			return optimizer


In [ ]:
def display_model(model):
	# Display a summary of the model architecture
	model.summary(expand_nested=True)

	# Display model architecture with layer shapes and trainable parameters
	tfk.utils.plot_model(model, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

In [ ]:
'''
Classifier training
'''
# Aug and optimizer params are taken from: https://arxiv.org/pdf/2110.09508
model = build_model(base_model=get_base_model())
base_model = model.get_layer(base_model_name)

if not LOAD_TRAINED_CLASSIFIER:
  # Compile the model with categorical cross-entropy loss and Adam optimizer
  model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=get_optimizer(is_fine_tuning=False, momentum=0.89), metrics=['accuracy'])
  display_model(model)
  
  # Fit the initial model
  print('\n\nFitting classifier\n\n')
  fit_history = fit_model(model)
  
  intermediate_val_acc = round(max(fit_history['val_accuracy']) * 100, 2)
  # Save intermediate model
  model_filename = f'vgg19-intermediateDONOTUSE-finetuned{len(based_model_layers_to_activate)}layers-{str(intermediate_val_acc)}-{datetime.now().strftime("%y%m%d_%H%M")}.keras'
  model.save(model_filename)

'''
Fine tuning
'''
# Enable fine tuning
enable_feature_extractor_layers(base_model, based_model_layers_to_activate)

# Compile the model with categorical cross-entropy loss and Adam optimizer
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=get_optimizer(is_fine_tuning=True, weight_decay=1e-5), metrics=['accuracy'])
display_model(model)

# Fit the initial finetuned model
print('\n\nFine tuning\n\n')
fit_history = fit_model(model)

# Calculate and print the best validation accuracy achieved
final_val_accuracy = round(max(fit_history['val_accuracy']) * 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

# Save the trained model to a file, including final accuracy in the filename
model_filename = f'vgg19-finetuned{len(based_model_layers_to_activate)}layers-{str(final_val_accuracy)}-{datetime.now().strftime("%y%m%d_%H%M")}.keras'
model.save(model_filename)

# Free memory by deleting the model instance
if FREE_MODEL:
  del model

In [ ]:
# Create figure and subplots for loss and accuracy
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 6))

# Plot loss for both re-trained and transfer learning models
ax1.plot(fit_history['loss'], alpha=0.3, color='#00008b', label='training loss', linestyle='--')
ax1.plot(fit_history['val_loss'], label='validation loss', alpha=0.8, color='#ffa500')
ax1.set_title('Categorical Crossentropy')
ax1.legend(loc='upper left')
ax1.grid(alpha=0.3)

# Plot accuracy for both re-trained and transfer learning models
ax2.plot(fit_history['accuracy'], alpha=0.3, color='#00008b', label='training accuracy', linestyle='--')
ax2.plot(fit_history['val_accuracy'], label='validation accuracy', alpha=0.8, color='#ffa500')
ax2.set_title('Accuracy')
ax2.legend(loc='upper left')
ax2.grid(alpha=0.3)

# Adjust layout to prevent label overlap and display the plots
plt.tight_layout()
plt.show()

## 👔 Load a trained model (if needed!)

In [ ]:
#model = tf.keras.models.load_model('KaggleEfficientNetV2L85.1241109_182031.keras')

## ✍🏿 Make evaluation

In [ ]:
##loss, acc = model.evaluate(X_test, y_test, verbose=2)
##print('Model, accuracy: {:5.2f}%'.format(100 * acc))

# Predict labels for the entire test set
predictions = model.predict(X_test, verbose=0)

# Display the shape of the predictions
print("Predictions Shape:", predictions.shape)

# Convert predictions to class labels
pred_classes = np.argmax(predictions, axis=-1)

# Extract ground truth classes
true_classes = np.argmax(y_test, axis=-1)

# Calculate and display test set accuracy as percentage
accuracy = accuracy_score(true_classes, pred_classes)
print(f'Accuracy score over the test set: {round(100 * accuracy, 2)}%')

# Calculate and display test set precision as percentage
precision = precision_score(true_classes, pred_classes, average='weighted')
print(f'Precision score over the test set: {round(100 * precision, 2)}%')

# Calculate and display test set recall as percentage
recall = recall_score(true_classes, pred_classes, average='weighted')
print(f'Recall score over the test set: {round(100 * recall, 2)}%')

# Calculate and display test set F1 score as percentage
f1 = f1_score(true_classes, pred_classes, average='weighted')
print(f'F1 score over the test set: {round(100 * f1, 2)}%')

# Compute the confusion matrix
cm = confusion_matrix(true_classes, pred_classes)

# Calculate the percentages for each element in the confusion matrix
cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

# Combine numbers and percentages into a single string for annotation
annot = np.array([f"{num}\n({percent:.2f}%)" for num, percent in zip(cm.flatten(), cm_percentage.flatten())]).reshape(cm.shape)

# Plot the confusion matrix with percentages
plt.figure(figsize=(10, 8))
sns.heatmap(cm_percentage.T, annot=annot, fmt='', xticklabels=list(labels.values()), yticklabels=list(labels.values()), cmap='Blues')
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.title('Confusion Matrix (Percentages)')
plt.show()


## 📊 Prepare Your Submission

To prepare your submission, create a `.zip` file that includes all the necessary code to run your model. It **must** include a `model.py` file with the following class:

```python
# file: model.py
class Model:
    def __init__(self):
        """Initialize the internal state of the model."""

    def predict(self, X):
        """Return a numpy array with the labels corresponding to the input X."""
```

The next cell shows an example implementation of the `model.py` file, which includes loading model weights from the `weights.keras` file and conducting predictions on provided input data. The `.zip` file is created and downloaded in the last notebook cell.

❗ Feel free to modify the method implementations to better fit your specific requirements, but please ensure that the class name and method interfaces remain unchanged.

In [ ]:
%%writefile model.py
import numpy as np

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl


class Model:
    def __init__(self):
        """
        Initialize the internal state of the model. Note that the __init__
        method cannot accept any arguments.

        The following is an example loading the weights of a pre-trained
        model.
        """
        self.neural_network = tfk.models.load_model('vgg19-finetuned3layers-98.54-241113_1106.keras')

    def predict(self, X):
        """
        Predict the labels corresponding to the input X. Note that X is a numpy
        array of shape (n_samples, 96, 96, 3) and the output should be a numpy
        array of shape (n_samples,). Therefore, outputs must no be one-hot
        encoded.

        The following is an example of a prediction from the pre-trained model
        loaded in the __init__ method.
        """
        preds = self.neural_network.predict(X)
        if len(preds.shape) == 2:
            preds = np.argmax(preds, axis=1)
        return preds

In [ ]:
from datetime import datetime
filename = f'submission_{datetime.now().strftime("%y%m%d_%H%M%S")}.zip'

# Add files to the zip command if needed
!zip {filename} model.py vgg19-finetuned3layers-98.54-241113_1106.keras